In [1]:
# Import Package
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

# Data Overview

In [2]:
# Reading data
Order = pd.read_excel("Order.xlsx")

In [3]:
# Seeing Top value
Order.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Unnamed: 17
0,1,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,0.000000
1,2,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,0.000000
2,3,CA-2019-138688,2019-06-12,2019-06-16,Second Class,DV-13045,United States,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,0.000000
3,4,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,430.909875
4,5,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,4.473600


# Data Quality Check

In [4]:
Order.isnull().sum()

Row ID             0
Order ID           0
Order Date         0
Ship Date          0
Ship Mode          0
Customer ID        0
Country/Region     0
Postal Code       11
Region             0
Product ID         0
Category           0
Sub-Category       0
Product Name       0
Sales              0
Quantity           0
Discount           0
Profit             0
Unnamed: 17        0
dtype: int64

In [5]:
Order['Postal Code'] = Order['Postal Code'].fillna(method='ffill')
Order['Postal Code'] = Order['Postal Code'].fillna(method='bfill')
Order['Postal Code'] = Order['Postal Code'].fillna(Order['Postal Code'].mean())

In [6]:
Order.isnull().sum()

Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Country/Region    0
Postal Code       0
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
Quantity          0
Discount          0
Profit            0
Unnamed: 17       0
dtype: int64

In [7]:
df = Order.drop(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Customer ID', 'Country/Region', 'Postal Code', 'Product ID', 'Sub-Category', 'Product Name', 'Unnamed: 17'], axis=1)

In [8]:
df.dtypes

Ship Mode     object
Region        object
Category      object
Sales        float64
Quantity       int64
Discount     float64
Profit       float64
dtype: object

In [9]:
df.describe()

,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000
mean,229.858001,3.789574,0.156203,28.656896
std,623.245101,2.225110,0.206452,234.260108
min,0.444000,1.000000,0.000000,-6599.978000
25%,17.280000,2.000000,0.000000,1.728750
50%,54.490000,3.000000,0.200000,8.666500
75%,209.940000,5.000000,0.200000,29.364000
max,22638.480000,14.000000,0.800000,8399.976000


# Preprocessing

In [10]:
# Clipping Data

Q1 = df['Profit'].quantile(0.25)
Q3 = df['Profit'].quantile(0.75)
IQR = Q3-Q1
Lwishker = Q1 - 1.5 * IQR
Uwishker = Q3 + 1.5 * IQR

df['Profit'] = df['Profit'].clip(Lwishker, Uwishker)

In [11]:
# Encoding
le = LabelEncoder()
df['Ship Mode'] = le.fit_transform(df['Ship Mode'])

In [12]:
# Dummies
dummies = pd.get_dummies(df[['Region', 'Category']], drop_first=True)
dummies.head()

,Region_East,Region_South,Region_West,Category_Office Supplies,Category_Technology
0,0,1,0,0,0
1,0,1,0,0,0
2,0,0,1,1,0
3,0,1,0,0,0
4,0,1,0,1,0


In [19]:
# Join Merged
df_merged = df.join(dummies)
df_final = df_merged.drop(['Region', 'Category'], axis=1)

,Ship Mode,Sales,Quantity,Discount,Profit,Region_East,Region_South,Region_West,Category_Office Supplies,Category_Technology
0,2,261.9600,2,0.00,41.913600,0,1,0,0,0
1,2,731.9400,3,0.00,70.816875,0,1,0,0,0
2,2,14.6200,2,0.00,6.871400,0,0,1,1,0
3,3,957.5775,5,0.45,-39.724125,0,1,0,0,0
4,3,22.3680,2,0.20,2.516400,0,1,0,1,0


# Model Selection

In [14]:
x = df_final.drop(['Profit'], axis='columns')
y = df_final['Profit']
y.loc[5]

14.169399999999996

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=90)

In [28]:
lr = LinearRegression()

# Modelling

In [16]:
# Modelling Section
lr = lr.fit(x_train,y_train)

In [17]:
# Model Prediction
print("Profit Prediction =", lr.predict([[4,1000,2,0,0,0,0,1,0]]))

Profit Prediction = [27.60437078]


C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


# Model Evaluation

In [18]:
# Modelling Evaluation
y_pred = lr.predict(x_test)
print("Mean Squared Error =",mean_squared_error(y_pred,y_test))

Mean Squared Error = 543.2974563373416
